<a href="https://colab.research.google.com/github/LucasAlbFar/Cobertura_Vacinal_BR/blob/main/notebooks/Gerar_csv_formatado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gerar CSV Formatado

## Funções para carregar arquivos via URL e gerar arquivos CSV, e salva-los no  na pasta 'dados_carregados'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def remover_linha_ou_coluna_indevida(data_frame: pd.DataFrame, tipo: str, lista_nomes: list):
  '''
    - Remover linha ou coluna, que esteja localizado na lista 'lista_nomes'
    - Parâmetros: Dataframe (tipo pandas) a ser atualizada
                  Tipo: identificador de 'linha' ou 'coluna' a ser removida
                  Lista de Nomes: Lista com o nome das linhas ou colunas a serem removidas
    - Retorno: Dataframe do Pandas                  
  '''
  
  if tipo == 'coluna':
    nome_ultima_columa = data_frame.iloc[:,-1].name.strip()
    while (nome_ultima_columa in lista_nomes):
      data_frame = data_frame.iloc[: , :-1] 
      nome_ultima_columa = data_frame.iloc[:,-1].name
  else:
    nome_ultima_linha = data_frame.iloc[-1].name
    while (nome_ultima_linha in lista_nomes):
      data_frame = data_frame.iloc[:-1 , :]
      nome_ultima_linha = data_frame.iloc[-1].name

  return data_frame

In [ ]:
def remover_cod_ibge_index(data_frame: pd.DataFrame, categoria: str):
  '''
    - Remover código IBGE contido no campo index do dataframe, de acordo com a categoria informada via parâmetro
    - Parâmetros: Dataframe (tipo pandas) a ser atualizada
                  Categoria do arquivo CSV contido na URL para remoção do código IBGE, podendo assumir os valores de:
                    * capital
                    * uf
                    * regiao
    - Retorno: Dataframe do Pandas                                  
  '''

  index_name = data_frame.index.name
  
  if (categoria == 'capital'):
    data_frame.index = [(capital[7:]) for capital in data_frame.index]
    data_frame.index.name = index_name
  elif (categoria == 'uf'):
    data_frame.index = [(estado[3:]) for estado in data_frame.index]
    data_frame.index.name = index_name
  elif (categoria == 'regiao'):
    data_frame.index = [(regiao[9:]) for regiao in data_frame.index]
    data_frame.index.name = index_name
  else:
    pass
    
  return data_frame

In [ ]:
def carregar_e_limpar_csv_para_df(url: str, categoria: str, skiprows: int, skipfooter: int):
  '''
    - Transformar o CSV contido na URL em pandas data_frame, removendo as últimas linhas e colunas que não trazem registros 
      pertinentes às análises a serem realizadas. Neste cenário, caso existam linhas ou colunas com os nomes de 'Total' ou 
      'Ignorados' serão removidos para função 'remover_linha_ou_coluna_indevida';
    - Determina a primeira coluna como índice do data_frame;
    - Substitui valores nulos por regra de interpolação determinada pelo Pandas;
    - Parâmetros: URL (tipo string) contendo o link do arquivo csv;
                  Categoria do arquivo CSV contido na URL para remoção do código IBGE, podendo assumir os valores de:
                    * capital
                    * uf
                    * regiao 
    - Retorno: Dataframe do Pandas
  '''

  lista_nomes_a_remover = ['Total', 'Ignorado', 'Ignorado/exterior']

  # Carregando dados da url parâmetro
  data_frame = pd.read_csv(url, encoding="ISO-8859-1", sep=";", skiprows=skiprows, skipfooter=skipfooter, thousands='.', decimal=',', engine='python') 

  # Removendo colunas contidas em lista_nomes_a_remover   
  data_frame = remover_linha_ou_coluna_indevida(data_frame,'coluna',lista_nomes_a_remover)

  # Definindo novo índice do dataframe
  index = data_frame.iloc[:,0].name
  data_frame = data_frame.set_index(index)

  # Removendo linhas contidas em lista_nomes_a_remover 
  data_frame = remover_linha_ou_coluna_indevida(data_frame,'linha',lista_nomes_a_remover)

  # Validando a existência de dados nulos e preenchendo esses valores com a função nativa de interpolação do pandas
  dados_nulos = data_frame.isnull().sum().sum()
  if (dados_nulos > 0):
    data_frame = data_frame.replace('-', np.nan)
    data_frame = data_frame.astype('float64') 
    data_frame = data_frame.fillna(data_frame.interpolate())
  
  # Removendo código IBGE Capital, Unidade da Federação ou Região (conforme parâmetro) do campo index
  if (categoria):
    data_frame = remover_cod_ibge_index(data_frame, categoria)

  return data_frame

In [ ]:
def buscar_web_para_csv():
  '''
    - Recuperar arquivo via URL e transforma-lo para um arquivo CSV, com as formatações executdas pela função 'carregar_e_limpar_csv_para_df'
  '''

  # Dicionários de dados contendo as URLS dos arquivos à serem carregadas e as categorias de cada arquivo
  urls = {
      'ano_por_imuno'                          : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/ano_por_imuno.csv', 'categoria':'', 'skiprows': 0, 'skipfooter': 1},
      'ano_por_capital'                        : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/ano_por_capital.csv', 'categoria':'capital', 'skiprows': 0, 'skipfooter': 1},
      'ano_por_uf'                             : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/ano_por_uf.csv', 'categoria':'uf', 'skiprows': 0, 'skipfooter': 1},
      'ano_por_regiao'                         : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/ano_por_regiao.csv', 'categoria':'regiao', 'skiprows': 0, 'skipfooter': 1},
      'imuno_por_capital'                      : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/imuno_por_capital.csv', 'categoria':'capital', 'skiprows': 0, 'skipfooter': 1},
      'imuno_por_uf'                           : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/imuno_por_uf.csv', 'categoria':'uf', 'skiprows': 0, 'skipfooter': 1},
      'imuno_por_regiao'                       : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/imuno_por_regiao.csv', 'categoria':'regiao', 'skiprows': 0, 'skipfooter': 1},
      'morbidade_tuberculose_1998_2007_cid10'  : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_turbeculose_1998_2007_CID10.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 3},
      'morbidade_tuberculose_1995_1997_cid09'  : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_turbeculose_1995_1997_CID09.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 3},
      'morbidade_tuberculose_2008_2021'        : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_turbeculose_2008_2021_CID10.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 6},
      'morbidade_meningite_1998_2007_cid10'    : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_meningite_1998_2007_CID10.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 3},
      'morbidade_meningite_1995_1997_cid09'    : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_meningite_1995_1997_CID09.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 3},
      'morbidade_meningite_2008_2021'          : {'url':'https://raw.githubusercontent.com/LucasAlbFar/Cobertura_Vacinal_BR/main/content/dados/morbidade_meningite_2008_2021_CID10.csv', 'categoria':'uf', 'skiprows': 4, 'skipfooter': 6}
  }

  # Execução da função 'carregar_e_limpar_csv_para_df' para cada URL do dicionário 'urls'
  for url_key, url_value in urls.items():
    file_name = '../content/dados_carregados/'+ url_key + '.csv'
    carregar_e_limpar_csv_para_df(url_value['url'], url_value['categoria'], url_value['skiprows'], url_value['skipfooter']).to_csv(file_name)

  print('Dados da web carregados para CSV no diretório /dados_carregados')


In [ ]:
buscar_web_para_csv()

Dados da web carregados para CSV no diretório /dados
